# <div style="color:white;display:fill;border-radius:5px;background-color:#0E2031;letter-spacing:0.5px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Conteúdo</p></div>     
   
**Data Preprocessing** 
- Bibliotecas
- Carregamento
- Glossário
- Dados Faltantes - KNN Inputer
- Engenharia de Dados

## <div style="color:white;display:fill;border-radius:5px;background-color:#0E2031;letter-spacing:0.5px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Data Preprocessing</p></div>  

## <div style="color:white;display:fill;border-radius:5px;background-color:#153656;letter-spacing:0.5px;overflow:hidden"><p style="padding:15px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Bibliotecas</p></div>

In [55]:
# Bibliotecas Basicas
import numpy as np
import pandas as pd
from tabulate import tabulate

## Bibliotecas para vizualização e estilização do notebook
from IPython.display import HTML
from plotly.offline import plot, iplot, init_notebook_mode
import plotly as py
from plotly import express as px
import plotly.graph_objs as go
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

# Bibliotecas para Preparação dos Dados
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from umap.umap_ import UMAP
from sklearn.manifold import TSNE

# Bibliotecas para o modelo Machine Learning
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from scipy.cluster import hierarchy
from sklearn.metrics import silhouette_score, davies_bouldin_score
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

In [56]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [57]:
def jupyter_table_settings():
    pd.set_option('display.expand_frame_repr', False )
    pd.set_option('display.max_columns', 50)
    pd.set_option('display.max_rows', 50)

In [58]:
jupyter_table_settings()

## <div style="color:white;display:fill;border-radius:5px;background-color:#153656;letter-spacing:0.5px;overflow:hidden"><p style="padding:15px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Carregamento</p></div>

In [59]:
data_raw = pd.read_csv('../data/CC_GENERAL.csv', encoding='iso-8859-1')
data_raw.shape

(8950, 18)

In [60]:
data = data_raw.copy()

In [61]:
# A base de dados é relativamente pequena e os dados são todos numéricos com exessão do CUST_ID, isso é um facilitador
# mais adiante no desenvolvimento do algoritimo de Machine Learning
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CUST_ID                           8950 non-null   object 
 1   BALANCE                           8950 non-null   float64
 2   BALANCE_FREQUENCY                 8950 non-null   float64
 3   PURCHASES                         8950 non-null   float64
 4   ONEOFF_PURCHASES                  8950 non-null   float64
 5   INSTALLMENTS_PURCHASES            8950 non-null   float64
 6   CASH_ADVANCE                      8950 non-null   float64
 7   PURCHASES_FREQUENCY               8950 non-null   float64
 8   ONEOFF_PURCHASES_FREQUENCY        8950 non-null   float64
 9   PURCHASES_INSTALLMENTS_FREQUENCY  8950 non-null   float64
 10  CASH_ADVANCE_FREQUENCY            8950 non-null   float64
 11  CASH_ADVANCE_TRX                  8950 non-null   int64  
 12  PURCHA

## <div style="color:white;display:fill;border-radius:5px;background-color:#153656;letter-spacing:0.5px;overflow:hidden"><p style="padding:15px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Glossário</p></div>

In [62]:
glossary_pt = [['Colunas', 'Significado'],
              ['CUST_ID', 'Identificador único do cliente'],
              ['BALANCE', 'Valor do saldo do cartão de crédito deixado na conta para fazer compras'],
              ['BALANCE_FREQUENCY', 'Frequência com que o saldo é atualizado, pontuação entre 0 e 1 (1 = Alto, 0 = Baixo)'],
              ['PURCHASES', 'Valor das compras feitas na conta'],
              ['ONEOFF_PURCHASES', 'Valor máximo de compra feito de uma só vez'],
              ['INSTALLMENTS_PURCHASES', 'Valor da compra parcelada'],
              ['CASH_ADVANCE', 'Dinheiro adiantado dado pelo usuário'],
              ['PURCHASES_FREQUENCY', 'Com que frequência as compras estão sendo feitas, pontuação entre 0 e 1 (1 = alta, 0 = baixa)'],
              ['ONEOFF_PURCHASES_FREQUENCY', 'Com que frequência as compras acontecem de uma só vez (1 = Alta, 0 = Baixa)'],
              ['PURCHASES_INSTALLMENTS_FREQUENCY', 'Com que frequência as compras parceladas estão sendo feitas (1 = Alta, 0 = Baixa)'],
              ['CASH_ADVANCE_FREQUENCY', 'Com que frequência o adiantamento em dinheiro está sendo pago'],
              ['CASH_ADVANCE_TRX', 'Número de transações feitas com "Cash in Advanced"'],
              ['PURCHASES_TRX', 'Número de transações de compra feitas'],
              ['CREDIT_LIMIT', 'Limite de Cartão de Crédito por usuário'],
              ['PAYMENTS', 'Valor do pagamento feito pelo usuário'],
              ['MINIMUM_PAYMENTS', 'Valor mínimo de pagamentos feitos pelo usuário'],
              ['PRC_FULL_PAYMENT', 'Porcentagem do pagamento integral pago pelo usuário'],
              ['TENURE', 'Prazo do serviço de cartão de crédito para o usuário']
              ]
#print(tabulate(glossary_pt, headers='firstrow', stralign='left', tablefmt='simple'))

## <div style="color:white;display:fill;border-radius:5px;background-color:#153656;letter-spacing:0.5px;overflow:hidden"><p style="padding:15px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Dados Faltantes</p></div>

In [63]:
data.isna().sum()

CUST_ID                               0
BALANCE                               0
BALANCE_FREQUENCY                     0
PURCHASES                             0
ONEOFF_PURCHASES                      0
INSTALLMENTS_PURCHASES                0
CASH_ADVANCE                          0
PURCHASES_FREQUENCY                   0
ONEOFF_PURCHASES_FREQUENCY            0
PURCHASES_INSTALLMENTS_FREQUENCY      0
CASH_ADVANCE_FREQUENCY                0
CASH_ADVANCE_TRX                      0
PURCHASES_TRX                         0
CREDIT_LIMIT                          1
PAYMENTS                              0
MINIMUM_PAYMENTS                    313
PRC_FULL_PAYMENT                      0
TENURE                                0
dtype: int64

In [64]:
X_knn = data.drop(['CUST_ID'], axis=1)
numerical_features = X_knn.select_dtypes(include = ['int64','float64']).columns.values

In [65]:
#Exemplo de linha com dados faltantes
data.iloc[8946]

CUST_ID                                 C19187
BALANCE                              19.183215
BALANCE_FREQUENCY                          1.0
PURCHASES                                300.0
ONEOFF_PURCHASES                           0.0
INSTALLMENTS_PURCHASES                   300.0
CASH_ADVANCE                               0.0
PURCHASES_FREQUENCY                        1.0
ONEOFF_PURCHASES_FREQUENCY                 0.0
PURCHASES_INSTALLMENTS_FREQUENCY      0.833333
CASH_ADVANCE_FREQUENCY                     0.0
CASH_ADVANCE_TRX                             0
PURCHASES_TRX                                6
CREDIT_LIMIT                            1000.0
PAYMENTS                            275.861322
MINIMUM_PAYMENTS                           NaN
PRC_FULL_PAYMENT                           0.0
TENURE                                       6
Name: 8946, dtype: object

In [66]:
num_imputer = KNNImputer()
num_imputer.fit(X_knn[numerical_features])
X_knn[numerical_features] = num_imputer.transform(X_knn[numerical_features])

In [67]:
# MINIMUM_PAYMENTS para o cliente C19187 encontrado pelo KNN foi 156.06
X_knn.loc[8946]

BALANCE                               19.183215
BALANCE_FREQUENCY                      1.000000
PURCHASES                            300.000000
ONEOFF_PURCHASES                       0.000000
INSTALLMENTS_PURCHASES               300.000000
CASH_ADVANCE                           0.000000
PURCHASES_FREQUENCY                    1.000000
ONEOFF_PURCHASES_FREQUENCY             0.000000
PURCHASES_INSTALLMENTS_FREQUENCY       0.833333
CASH_ADVANCE_FREQUENCY                 0.000000
CASH_ADVANCE_TRX                       0.000000
PURCHASES_TRX                          6.000000
CREDIT_LIMIT                        1000.000000
PAYMENTS                             275.861322
MINIMUM_PAYMENTS                     156.060050
PRC_FULL_PAYMENT                       0.000000
TENURE                                 6.000000
Name: 8946, dtype: float64

In [68]:
data_new = pd.concat([data[['CUST_ID']], X_knn],axis=1)
data_new.shape

(8950, 18)

In [69]:
data_new.duplicated().sum()

0

## <div style="color:white;display:fill;border-radius:5px;background-color:#153656;letter-spacing:0.5px;overflow:hidden"><p style="padding:15px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Engenharia de Dados</p></div>

In [51]:
data_new['PROFIT'] = (data_new.PAYMENTS - data_new.PURCHASES) #CASH_ADVANCE como calcular o pagamento?
data_new['LIMIT_EXCEED'] = data_new.apply(lambda x: 1 if (x.PURCHASES > x.CREDIT_LIMIT) else 0, axis=1)